## Chat GPT 4 Build

In [7]:
from collections import Counter, defaultdict

# Load the content of the provided text files
with open("five_letter_words.txt", "r") as file:
    five_letter_words = [word.strip().upper() for word in file.readlines()]

with open("OldWords.txt", "r") as file:
    OldWords = [word.strip().upper() for word in file.readlines()]

def remove_words(five_letter_words, OldWords):
    """Removes all words from `five_letter_words` that are also in `OldWords`."""
    return [word for word in five_letter_words if word not in OldWords]

five_letter_words = remove_words(five_letter_words, OldWords)

def count_letter_frequency(words_list):
    """Counts the frequency of each letter in the given list of words."""
    letter_frequency = Counter("".join(words_list))
    return sorted(letter_frequency.items(), key=lambda x: x[1], reverse=True)

top_common_letters = count_letter_frequency(five_letter_words)

def filter_words_based_on_letters(words_list, top_common_letters, num_letters=5):
    """Filters the words based on the most common letters and extracts unique letters."""
    filtered_words = words_list.copy()
    for letter, _ in top_common_letters[:num_letters]:
        filtered_words = [word for word in filtered_words if letter in word]
    
    unique_letters = set("".join(filtered_words))
    return filtered_words, list(unique_letters)

filtered_words, unique_letters = filter_words_based_on_letters(five_letter_words, top_common_letters)

def most_common_combinations(words_list, starting_letters, top_n=5):
    """Finds the most common two-letter combinations in the given list of words."""
    two_letter_combinations = Counter()
    for word in words_list:
        for i in range(len(word)):
            for j in range(i+1, len(word)):
                if word[i] in starting_letters and word[j] in starting_letters:
                    combo = tuple(sorted([word[i], word[j]]))
                    two_letter_combinations[combo] += 1
    return two_letter_combinations.most_common(top_n)


def filter_words_by_locked_positions(words_list, locked_positions):
    """Filters words that have the locked letters in the locked positions."""
    for position, letter in locked_positions.items():
        words_list = [word for word in words_list if len(word) >= position and word[position-1] == letter]
    return words_list

def filter_words_by_excluded_positions(words_list, excluded_positions):
    """Filters out words that have the specified letters in the excluded positions."""
    for position, letter in excluded_positions.items():
        words_list = [word for word in words_list if len(word) < position or word[position-1] != letter]
    return words_list


def filter_words_based_on_parameters_updated(words_list, letters_included, letters_not_included, locked_positions, excluded_positions):
    """Filters the word list based on the specified parameters."""
    # Filter by letters included
    for letter in letters_included:
        words_list = exclude_words_with_letters(words_list, letters_not_included)
    
    # Filter by letters not included
    words_list = exclude_words_with_letters(words_list, letters_not_included)
    
    # Filter by locked positions
    words_list = filter_words_by_locked_positions(words_list, locked_positions)
    
    # Filter by excluded positions
    words_list = filter_words_by_excluded_positions(words_list, excluded_positions)
    
    return words_list

# Renaming and defining the function for excluding words with certain letters
def exclude_words_with_letters(words_list, excluded_letters):
    """Excludes words that contain any of the letters marked as incorrect by the user."""
    for letter in excluded_letters:
        words_list = [word for word in words_list if letter not in word]
    return words_list

def exclude_words_with_letters(words_list, excluded_letters):
    """Excludes words that contain any of the letters marked as incorrect by the user."""
    for letter in excluded_letters:
        words_list = [word for word in words_list if letter not in word]
    return words_list


## Recomendations for next word

In [8]:
def compute_letter_scores(words_list):
    """Computes the frequency scores for each letter in the words list."""
    letter_frequency = Counter("".join(words_list))
    total_letters = sum(letter_frequency.values())
    
    # Normalize the letter scores
    letter_scores = {letter: freq / total_letters for letter, freq in letter_frequency.items()}
    return letter_scores

def compute_combination_scores(words_list, unique_letters):
    """Computes the scores for two-letter combinations in the words list."""
    two_letter_combinations = most_common_combinations(words_list, unique_letters, top_n=None)
    total_combinations = sum([count for _, count in two_letter_combinations])
    
    # Normalize the combination scores
    combination_scores = {combo: count / total_combinations for combo, count in two_letter_combinations}
    return combination_scores

def top_n_recommended_words_with_scores(words_list, top_common_letters, unique_letters, n=20, repeat_multiplier=0.5):
    """Recommends the top n optimal words based on letter and combination scores and returns the breakdown."""
    letter_scores = compute_letter_scores(words_list)
    combination_scores = compute_combination_scores(words_list, unique_letters)
    
    word_scores = {}
    word_letter_scores = {}
    word_combination_scores = {}
    for word in words_list:
        # Letter score
        word_letter_score = sum([letter_scores.get(letter, 0) for letter in word])
        word_letter_scores[word] = round(word_letter_score, 3)
        
        # Combination score
        word_combination_score = 0
        for i in range(len(word)):
            for j in range(i+1, len(word)):
                combo = tuple(sorted([word[i], word[j]]))
                word_combination_score += combination_scores.get(combo, 0)
        word_combination_scores[word] = round(word_combination_score, 3)
        
        # Apply the multiplier if the word has repeated letters
        multiplier = repeat_multiplier if len(word) > len(set(word)) else 1.0
                
        # Total score
        word_scores[word] = round((word_letter_score + word_combination_score) * multiplier, 3)
    
    # Get the top n words with the highest scores
    sorted_words = sorted(word_scores, key=word_scores.get, reverse=True)[:n]
    headers = ["Word", "Total Score", "Letter Score", "Combination Score"]
    return headers, sorted_words, [word_scores[word] for word in sorted_words], [word_letter_scores[word] for word in sorted_words], [word_combination_scores[word] for word in sorted_words]


## Graphical interphase

In [9]:
import tkinter as tk
from tkinter import StringVar, ttk
from functools import partial
from tkinter import ttk, Text, Scrollbar

# Initialize the main window
root = tk.Tk()
root.title("Wordle Solver")

# Variables to store user's choices
letters_included = []
letters_not_included = []
locked_positions = {}
excluded_positions = {}
submitted_word = "" 
letter_states = {}  # {position: "locked"/"excluded"/"wrong_place"}


# Command functions for buttons

# Define a function to reset the button color
def reset_button_color(button):
    button.config(bg='SystemButtonFace')

# Modify the existing command functions to reset the colors of the other buttons
def create_lock_command(lock_button, exclude_button, wrong_place_button, letter, position, letter_state):
    def command():
        reset_button_color(exclude_button)
        reset_button_color(wrong_place_button)
        lock_in_place(lock_button, letter, position)
        letter_state['locked'] = letter
    return command

def create_exclude_command(lock_button, exclude_button, wrong_place_button, letter, position, letter_state):
    def command():
        reset_button_color(lock_button)
        reset_button_color(wrong_place_button)
        exclude_letter(exclude_button, letter, position)
        letter_state['locked'] = letter
    return command

def create_wrong_place_command(lock_button, exclude_button, wrong_place_button, letter, position, letter_state):
    def command():
        reset_button_color(lock_button)
        reset_button_color(exclude_button)
        mark_wrong_place(wrong_place_button, letter, position)
        letter_state['locked'] = letter
    return command




def lock_in_place(button, letter, position):
    global locked_positions  # Declare the variable as global
    locked_positions[position] = letter
    #button.config(state=tk.DISABLED)
    button.config(bg='green')

    # Add the print statements here
    print("After locking a letter:")
    print("Letters not included:", letters_not_included)
    print("Locked positions:", locked_positions)
    print("Excluded positions:", excluded_positions)

def do_not_include(button, letter):
    global letters_not_included  # Declare the variable as global
    letters_not_included.append(letter)
    #button.config(state=tk.DISABLED)
    button.config(bg='red')

    # Add the print statements here
    print("After excluding a letter:")
    print("Letters not included:", letters_not_included)
    print("Locked positions:", locked_positions)
    print("Excluded positions:", excluded_positions)

def wrong_place(button, letter, position):
    global letters_not_included, excluded_positions  # Declare the variables as global
    excluded_positions[position] = letter
    letters_not_included.append(letter)
    #button.config(state=tk.DISABLED)
    button.config(bg='yellow')

    # Add the print statements here
    print("After marking a letter as in the wrong place:")
    print("Letters not included:", letters_not_included)
    print("Locked positions:", locked_positions)
    print("Excluded positions:", excluded_positions)


def display_buttons(position):
    if letter_states[position] == "locked":
        # Change the appearance of the lock button, reset others
        ...
    elif letter_states[position] == "excluded":
        # Change the appearance of the exclude button, reset others
        ...
    elif letter_states[position] == "wrong_place":
        # Change the appearance of the wrong place button, reset others
        ...

def exclude_letter(button, letter, position):
    # Logic for marking the letter as excluded
    # Update the global variable
    letters_not_included.append(letter)
    # Update the UI, if necessary
    button.config(bg='red')  # As an example, change the background to red

def mark_wrong_place(button, letter, position):
    # Logic for marking the letter as in the wrong place
    excluded_positions[position] = letter
    # Update the UI
    button.config(bg='yellow')  # Change the background to yellow, as an example




from functools import partial

def display_letters():
    print("display_letters function called!")  # Debug print

    global submitted_word  # Declare it as global
    submitted_word = word_entry.get().upper()  # Retrieve the word from the entry widget

    # Initialize the local state for each letter
    letters_states = [{} for _ in submitted_word]

    print("display_letters function called!")
    for widget in root.winfo_children():
        widget.grid_forget()  # Clear the previous widgets

    # Create all buttons first
    lock_buttons = []
    exclude_buttons = []
    wrong_place_buttons = []

    for i, letter in enumerate(submitted_word):
        ttk.Label(root, text=letter, width=5).grid(row=1, column=i+1)

        lock_button = tk.Button(root, text="Lock")
        lock_button.grid(row=2, column=i+1)
        lock_buttons.append(lock_button)

        exclude_button = tk.Button(root, text="Exclude")
        exclude_button.grid(row=3, column=i+1)
        exclude_buttons.append(exclude_button)

        wrong_place_button = tk.Button(root, text="Wrong Place")
        wrong_place_button.grid(row=4, column=i+1)
        wrong_place_buttons.append(wrong_place_button)

    # Now configure their commands
    for i, letter in enumerate(submitted_word):
        lock_buttons[i].config(command=create_lock_command(lock_buttons[i], exclude_buttons[i], wrong_place_buttons[i], letter, i+1, letters_states[i]))
        exclude_buttons[i].config(command=create_exclude_command(lock_buttons[i], exclude_buttons[i], wrong_place_buttons[i], letter, i+1, letters_states[i]))
        wrong_place_buttons[i].config(command=create_wrong_place_command(lock_buttons[i], exclude_buttons[i], wrong_place_buttons[i], letter, i+1, letters_states[i]))

    submit_btn.grid(row=0, column=2, padx=10, pady=10)
    solve_btn.grid(row=5, column=1, columnspan=5, pady=10)
    output_text.grid(row=7, column=0, columnspan=6, padx=10, pady=10)
    scrollbar.grid(row=7, column=6, sticky='ns')

    def solve_wordle_local():
        update_global_state()
        solve_wordle()

    solve_btn.config(command=solve_wordle_local)




def update_global_state(letters_states):
    global letters_not_included, locked_positions, excluded_positions
    locked_positions = {}
    excluded_positions = {}
    letters_not_included = []

    for i, state in enumerate(letters_states):
        if 'locked' in state:
            locked_positions[i+1] = state['locked']
        if 'excluded' in state:
            letters_not_included.append(state['excluded'])
        if 'wrong_place' in state:
            excluded_positions[i+1] = state['wrong_place']


def solve_wordle_local():
    update_global_state(letters_states)
    solve_wordle()

def solve_wordle():
    print(f"Processing the submitted word: {submitted_word}")  # Debug print
    word = submitted_word

    print("Solve Wordle function called!")  # Debug print
    print(f"Processing the submitted word: {submitted_word}")

    global letters_included, letters_not_included, locked_positions, excluded_positions
    
    # Get the word from the entry
    word = submitted_word.upper()
    
    # Filter the words
    filtered_words_updated = filter_words_based_on_parameters_updated(five_letter_words, letters_included, letters_not_included, locked_positions, excluded_positions)
    print("Filtered words after processing:", filtered_words_updated[:10])


    # Recommend the next word
    top_words, top_total_scores, top_letter_scores, top_combination_scores, headers = top_n_recommended_words_with_scores(filtered_words_updated, top_common_letters, unique_letters)
    
    # Prepare the output to be displayed
    output_content = []
    output_content = [f"{word}, {total_score}, {letter_score}" for word, total_score, letter_score in zip(top_words, top_total_scores, top_letter_scores)]

    '''
    output_content.append(f"A {top_words[0]}, {top_total_scores[0]}, {top_letter_scores[0]}")
    output_content.append(f"B {top_words[1]}, {top_total_scores[1]}, {top_letter_scores[1]}")
    output_content.append(f"C {top_words[2]}, {top_total_scores[2]}, {top_letter_scores[2]}")
    output_content.append(f"D {top_words[3]}, {top_total_scores[3]}, {top_letter_scores[3]}")
    '''
    
    output_content.append("\nRecommended next word:")
    for score, word in zip(top_total_scores[4:], top_words[4:]):
        output_content.append(f"{score}, {word}")

    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, '\n'.join(output_content))

    print("Solve Wordle function finished!")  # Debug print at the end


# Function to clear the output and reset the input
def clear_and_reset():
    global submitted_word
    submitted_word = ""  # Reset the submitted word
    for widget in root.winfo_children():
        widget.grid_forget()
    create_initial_ui()

def submit_word():
    print("Submit word function started")
    global submitted_word
    submitted_word = next_word_entry.get().upper()
    print(f"Submitted word: {submitted_word}")  # Debugging
    display_letters()
    print("Submit word function ended")

# Entry field for the 5-letter word
word_entry = tk.Entry(root, width=15)
word_entry.grid(row=0, column=1, padx=10, pady=10)

submit_btn = tk.Button(root, text="Submit Word", command=display_letters)
submit_btn.grid(row=0, column=2, padx=10, pady=10)

# Button to solve Wordle
solve_btn = tk.Button(root, text="Solve Wordle", command=solve_wordle)
solve_btn.grid(row=5, column=1, columnspan=5, pady=10)

# Text widget to display multiple lines of output
output_text = Text(root, width=50, height=15)
output_text.grid(row=7, column=0, columnspan=6, padx=10, pady=10)
scrollbar = Scrollbar(root, command=output_text.yview)
scrollbar.grid(row=7, column=6, sticky='nsew')
output_text.config(yscrollcommand=scrollbar.set)

# Entry for the next word
word_entry = ttk.Entry(root, width=15)
word_entry.grid(row=0, column=1, padx=10, pady=10)

# Button to submit the next word
submit_button = ttk.Button(root, text="Submit Word", command=display_letters)
submit_button.grid(row=0, column=2, padx=10, pady=10)

root.mainloop()


display_letters function called!
display_letters function called!
After locking a letter:
Letters not included: []
Locked positions: {1: 'Q'}
Excluded positions: {}
After locking a letter:
Letters not included: ['Q']
Locked positions: {1: 'Q', 2: 'W'}
Excluded positions: {1: 'Q'}
After locking a letter:
Letters not included: ['Q', 'W']
Locked positions: {1: 'Q', 2: 'W', 3: 'E'}
Excluded positions: {1: 'Q'}
After locking a letter:
Letters not included: ['Q', 'W', 'E']
Locked positions: {1: 'Q', 2: 'W', 3: 'E', 4: 'R'}
Excluded positions: {1: 'Q'}
After locking a letter:
Letters not included: ['Q', 'W', 'E', 'R']
Locked positions: {1: 'Q', 2: 'W', 3: 'E', 4: 'R', 5: 'T'}
Excluded positions: {1: 'Q'}


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\JonasCarlsson\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\JonasCarlsson\AppData\Local\Temp\ipykernel_23696\3861790167.py", line 165, in solve_wordle_local
    update_global_state()
  File "C:\Users\JonasCarlsson\AppData\Local\Temp\ipykernel_23696\3861790167.py", line 179, in update_global_state
    for i, state in enumerate(letters_states):
                              ^^^^^^^^^^^^^^
NameError: name 'letters_states' is not defined


KeyboardInterrupt: 

## Play the game

In [ ]:
# Set the variables
letters_not_included = []# ['A', 'R', 'O', 'E', 'U', 'T', 'C', 'K'] #, 'W', 'N', 'T', 'M']
letters_included = []#['N', 'I','S']
locked_positions = {}#{ 1: 'S', 2: 'N', 3: 'I' }
excluded_positions = {}#{4: 'S', 5: 'S'}#{1: 'D', 5: 'P'}

# Set the repeat_multiplier
repeat_multiplier_value = 0.5

# Filter the words based on the specified parameters using the updated function
filtered_words_updated = filter_words_based_on_parameters_updated(five_letter_words, letters_included, letters_not_included, locked_positions, excluded_positions)

# Recommend the next word
headers, top_words, top_total_scores, top_letter_scores, top_combination_scores = top_n_recommended_words_with_scores(filtered_words_updated, top_common_letters, unique_letters, repeat_multiplier=repeat_multiplier_value)

# Display the results
print("Filtered words (first 10):", filtered_words_updated[:10])
print("Recommended next word:")
print(headers)
list(zip(top_words, top_total_scores, top_letter_scores, top_combination_scores))


Filtered words (first 10): ['AAHED', 'AALII', 'AAPAS', 'AARGH', 'AARTI', 'ABACA', 'ABACI', 'ABACS', 'ABAFT', 'ABAHT']
Recommended next word:
['Word', 'Total Score', 'Letter Score', 'Combination Score']


[('AEROS', 1.279, 0.43, 0.849),
 ('AROSE', 1.279, 0.43, 0.849),
 ('SOARE', 1.279, 0.43, 0.849),
 ('AESIR', 1.014, 0.419, 0.595),
 ('ARISE', 1.014, 0.419, 0.595),
 ('RAISE', 1.014, 0.419, 0.595),
 ('REAIS', 1.014, 0.419, 0.595),
 ('SERAI', 1.014, 0.419, 0.595),
 ('SERIA', 1.014, 0.419, 0.595),
 ('ARLES', 1.005, 0.411, 0.595),
 ('EARLS', 1.005, 0.411, 0.595),
 ('LAERS', 1.005, 0.411, 0.595),
 ('LARES', 1.005, 0.411, 0.595),
 ('LASER', 1.005, 0.411, 0.595),
 ('LEARS', 1.005, 0.411, 0.595),
 ('RALES', 1.005, 0.411, 0.595),
 ('REALS', 1.005, 0.411, 0.595),
 ('SERAL', 1.005, 0.411, 0.595),
 ('ARETS', 1.004, 0.409, 0.595),
 ('ASTER', 1.004, 0.409, 0.595)]